<a href="https://colab.research.google.com/github/jeguns/EP7192/blob/main/Capitulo_1/Unidad_1_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de paquetes

In [ ]:
!pip install pywaffle > /dev/null 2>&1
!pip install squarify > /dev/null 2>&1
!pip install ydata-profiling  > /dev/null 2>&1
!pip install skimpy > /dev/null 2>&1

In [ ]:
import pandas as pd
import numpy as np
import statistics as stats
from scipy.stats import iqr, variation, skew
import math
import matplotlib.pyplot as plt
from pywaffle import Waffle
import squarify
import seaborn as sns
from skimpy import skim
from ydata_profiling import ProfileReport

# Lectura de datos

In [ ]:
datos = pd.read_csv('Salud.csv')

In [ ]:
datos.head(5)

In [ ]:
datos.tail(5)

# Medidas de tendencia central

## Media

### Ejemplo 1

Interpretar la media aritmética de la edad

In [ ]:
datos['Edad'].mean()

In [ ]:
np.mean(datos['Edad'])

In [ ]:
float(datos['Edad'].mean())

In [ ]:
float(np.mean(datos['Edad']))

In [ ]:
stats.mean(datos['Edad'])

La edad promedio de los pacientes es de 46 años.

### Ejemplo 2

Interpretar la presión sistólica media de los pacientes mayores de 50 años.

In [ ]:
 datos.loc[datos['Edad'] > 50].agg(Media=('Presion_sistolica', 'mean'))

In [ ]:
(datos
 .loc[datos['Edad'] > 50]
 .agg(Media=('Presion_sistolica', 'mean')))

La presión sistólica promedio de los pacientes mayores de 50 años es de 130 mmHg.

## Mediana

### Ejemplo 3

Interpretar la mediana del IMC

In [ ]:
datos['IMC'].median()

In [ ]:
np.median(datos['IMC'])

In [ ]:
float(np.median(datos['IMC']))

In [ ]:
stats.median(datos['IMC'])

Al menos la mitad de las personas tiene un IMC menor o igual a 19.2.

### Ejemplo 4

Interpretar la mediana de la presión sistólica para las personas que son sedentarias (menos de
30 minutos de ejercicios a la semana) y las que no lo son.

In [ ]:
datos = datos.assign(
    Sedentario = np.where(datos["Minutos_ejercicio"] < 30, "Sí", "No"))

In [ ]:
datos.groupby("Sedentario", as_index=False).agg(Medianas=("Presion_sistolica", "median"))

In [ ]:
(datos
 .groupby("Sedentario", as_index=True)
 .agg(Medianas=("Presion_sistolica", "median")))

Al menos la mitad de las personas sedentarias presenta una presión sistólica de como máximo
137.5 mmHg (¡elevada!). Por otro lado, al menos el 50% de las personas que no son sedentarias
tiene una presión sistólica menor o igual a 121 mmHg (casi en el rango normal).

## Moda

### Ejemplo 5

Interpretar la moda de la presión sistólica

In [ ]:
datos["Presion_sistolica"].mode()

La presión sistólica más frecuente es de 121 mHg.

### Ejemplo 6

Interpretar la moda de la edad

In [ ]:
datos["Edad"].mode()

Las edades más frecuentes de los pacientes son 36 y 59 años.

### Ejemplo 7

Intepretar la moda del tiempo semanal de ejercicio de los pacientes sedentarios

In [ ]:
(datos
 .loc[datos["Sedentario"] == "Sí", "Minutos_ejercicio"]
 .mode())

El tiempo de ejercicios más frecuente entre los pacientes sedentarios es de 15 minutos.

## Medidas de posición

### Ejemplo 8

Interpretar el percentil 41 de la eda

In [ ]:
datos['Edad'].quantile(0.41)

In [ ]:
float(datos['Edad'].quantile(0.41))

Al menos el 41% de los pacientes tiene 42 años de edad o menos.

### Ejemplo 9

Interpretar los percentiles 12 y 74 de los tiempos semanales de ejercicio de las personas no
sedentarias

In [ ]:
(datos
 .loc[datos["Sedentario"] == "No", "Minutos_ejercicio"]
 .quantile([0.12, 0.74]))

Al menos el 12% de los pacientes no sedentarios realiza como máximo 61.44 minutos de ejercicio
a la semana, mientras que al menos el 74% realiza hasta 240.76 minutos semanales de actividad
física.

### Ejemplo 10

Interpretar los cuartiles del IMC de las personas adultas mayores (60 años a más)

In [ ]:
(datos
 .loc[datos["Edad"] > 60, "IMC"]
 .quantile([0.25, 0.50, 0.75]))

Al menos el 25% de los pacientes tiene un IMC igual o inferior a 19.425, mientras que como
máximo el 50% tiene un IMC igual o inferior a 20. Además, hasta el 75% de los pacientes
presenta un IMC igual o inferior a 20.925.

### Ejemplo 11

¿Cuál es el tiempo máximo de ejercicio semanal que realiza un paciente joven (menor de 30
años) para estar dentro del 20% que menos ejercicio realiza?

In [ ]:
(datos
 .loc[datos["Edad"] < 30, "Minutos_ejercicio"]
 .quantile([0.2]))

58 minutos semanales es el tiempo máximo de ejercicio que realiza un paciente joven (menor
de 30 años) para estar dentro del 20% que menos ejercicio realiza.

# Medidas de dispersión

## Rango

### Ejemplo 12

Interpretar el rango de la edad

In [ ]:
np.ptp(datos['Edad'])

In [ ]:
float(np.ptp(datos['Edad']))

La amplitud de la edad es de 49 años.

### Ejemplo 13

Interpretar el rango del IMC para cada grupo de personas según su nivel de actividad física
(sedentario / no sedentario).

In [ ]:
(datos
 .groupby("Sedentario", as_index=True)
 .agg(r=("IMC", lambda x: np.ptp(x))))

La amplitud del IMC de las personas sedentarias es de 4.7 puntos, mientras que para las no
sedentarias es de 9.5 puntos.

## Rango intercuartil

### Ejemplo 14

Interpretar el rango intercuartil de la edad

In [ ]:
float(datos["Edad"].quantile(0.75) - datos["Edad"].quantile(0.25))

In [ ]:
float(iqr(datos["Edad"]) )

La amplitud del 50% central de las edades es de 22.25 años.

### Ejemplo 15

Interpretar el rango intercuartil del IMC para cada grupo de personas según su nivel de
actividad física (sedentario / no sedentario).

In [ ]:
(datos
 .groupby("Sedentario", as_index=True)
 .agg(ric=("IMC", lambda x: iqr(x))))

La amplitud del 50% central de los datos de IMC de las personas sedentarias es de 1.7 puntos,
mientras que para las no sedentarias es de 2.6 puntos.

## Varianza

### Ejemplo 16

Calcular la varianza del tiempo semanal de ejercicios.

In [ ]:
datos['Minutos_ejercicio'].var()

La varianza del tiempo semanal de ejercicios es de 8183 $minutos^2$

## Desviación estándar

### Ejemplo 17

Interpretar la desviación estándar del tiempo semanal de ejercicios.

In [ ]:
datos['Minutos_ejercicio'].std()

In [ ]:
math.sqrt(datos['Minutos_ejercicio'].var())

En promedio, el tiempo semanal de ejercicios se desvía 90.5 minutos respecto su media.

## Coeficiente de variabilidad

### Ejemplo 18

¿Qué variable presenta mayor variabilidad: el IMC o la presión sistólica?

In [ ]:
pd.DataFrame({
    "s_imc": [datos["IMC"].std()],
    "s_pres": [datos["Presion_sistolica"].std()],
    "m_imc": [datos["IMC"].mean()],
    "m_pres": [datos["Presion_sistolica"].mean()],
    "cv_imc": [variation(datos["IMC"]) * 100],
    "cv_pres": [variation(datos["Presion_sistolica"]) * 100]})

El IMC presenta mayor variabilidad (cv = 10.61%) que la presión sistólica (cv = 10.23%).

### Ejemplo 19

Los pacientes se dividen en 3 grupos: joven (menor de 30 años),adulto (de 30 a 59 años) y
adulto mayor (de 60 a más años). ¿En qué grupo se observa mayor variabilidad en el tiempo
semanal de ejercicios?

In [ ]:
datos = datos.assign(
    Grupo_Edad = pd.cut(
        datos["Edad"],
        bins=[-float("inf"), 30, 60, float("inf")],
        labels=["Joven", "Adulto", "Adulto mayor"],
        right=False  # para que 30 esté incluido en "Adulto"
))
datos.head(5)

In [ ]:
(datos
 .groupby("Grupo_Edad", as_index=False)
 .agg(s=("Minutos_ejercicio", "std"),
      m=("Minutos_ejercicio", "mean"),
      cv=("Minutos_ejercicio", lambda x: variation(x)*100))
)

El grupo con mayor variabilidad en el tiempo semanal de ejercicios es el de adultos mayores.

# Medidas de asimetría

## Coeficiente de asimetría de Fisher Pearson

### Ejemplo 20

Interpretar el coeficiente de asimetría de Fisher Pearson para cada variable.

In [ ]:
datos.select_dtypes(include='number').apply(lambda x: skew(x, nan_policy='omit')).to_frame().T

Las cuatro variables presentan distribuciones que tienden a la simetría, siendo la edad la que
más se acerca a la simetría y el IMC la que más se aleja.

## Coeficiente de asimetría de Bowley

### Ejemplo 21

Interpretar el coeficiente de asimetría de Bowley para cada variable.

In [ ]:
def bowley(x):
    q1 = x.quantile(0.25)
    q2 = x.quantile(0.50)
    q3 = x.quantile(0.75)
    return (q3 + q1 - 2 * q2) / (q3 - q1) if (q3 - q1) != 0 else None

datos.select_dtypes(include='number').apply(bowley).to_frame().T

Las cuatro variables presentan distribuciones que tienden a la simetría en el 50% central de los
datos, siendo el IMC la que más se acerca a la simetría y la edad la que más se aleja. ¿Es una
contradicción respecto a la interpretación con el coeficiente de asimetría de Fisher Pearson?

# Tablas de frecuencia


## Tablas de frecuencia para variables cualitativas

In [ ]:
tabla = (
    datos['Sedentario']
    .value_counts(dropna=False)
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Categoría', 'count': 'Frecuencia'}))


tabla['Frecuencia_relativa'] = tabla['Frecuencia'] / tabla['Frecuencia'].sum()

tabla

## Tablas de frecuencia para variables cuantitativas discretas




In [ ]:
tabla = (
    datos.loc[datos['Edad'] > 60]['Edad']
    .value_counts(dropna=False)
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Valor', 'count': 'Frecuencia'})
)

tabla['Frecuencia_relativa'] = tabla['Frecuencia'] / tabla['Frecuencia'].sum()
tabla['Frecuencia_acumulada'] = tabla['Frecuencia'].cumsum()
tabla['Frecuencia_relativa_acumulada'] = tabla['Frecuencia_relativa'].cumsum()
tabla

## Tablas de frecuencia para variables cuantitativas continuas


In [ ]:
intervalos = pd.cut(datos['Edad'], bins=5)

tabla = (
    intervalos
    .value_counts(sort=False)
    .reset_index()
    .rename(columns={'index': 'Intervalo', 'count': 'Frecuencia'})
)

tabla['Frecuencia_relativa'] = tabla['Frecuencia'] / tabla['Frecuencia'].sum()
tabla['Frecuencia_acumulada'] = tabla['Frecuencia'].cumsum()
tabla['Frecuencia_relativa_acumulada'] = tabla['Frecuencia_relativa'].cumsum()
tabla

# Gráficas

## Gráficas para variables cualitativas

### Gráfico de barras

In [ ]:
conteo = datos['Sedentario'].value_counts().sort_index().reset_index()
conteo.columns = ['Sedentario', 'n']

fig, ax = plt.subplots(figsize=(6, 4))
barras = ax.bar(conteo['Sedentario'], conteo['n'], color='steelblue')

for i, row in conteo.iterrows():
    ax.text(row['Sedentario'], row['n'] / 2, str(row['n']),
            ha='center', va='center', color='white', fontweight='bold')

ax.set_title("Distribución de pacientes según su actividad física")
ax.set_xlabel("Actividad física")
ax.set_ylabel("Cantidad de pacientes")
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
barras = ax.barh(conteo['Sedentario'], conteo['n'], color='forestgreen')

for i, row in conteo.iterrows():
    ax.text(row['n'] / 2, row['Sedentario'], str(row['n']),
            va='center', ha='center', color='white', fontweight='bold')

ax.set_title("Distribución de pacientes según su actividad física")
ax.set_xlabel("Cantidad de pacientes")
ax.set_ylabel("Actividad física")
plt.tight_layout()
plt.show()

### Gráfico circular

In [ ]:
conteo = (
    datos['Sedentario']
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Sedentario', 'Sedentario': 'n'})
)

conteo['porcentaje'] = round(conteo['count'] / conteo['count'].sum() * 100, 1)
conteo['etiqueta'] = conteo['n'] + ": " + conteo['porcentaje'].astype(str) + "%"
conteo

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(
    conteo['count'],
    labels=conteo['etiqueta'],
    colors=plt.cm.Set2.colors,  # paleta suave
    startangle=90,
    wedgeprops=dict(width=0.7)  # anillo delgado como `coord_polar`
)

ax.set_title("Distribución de pacientes según su actividad física")
plt.tight_layout()
plt.show()

### Gráfico de waffle

In [ ]:
conteo = (
    datos['Sedentario']
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Sedentario', 'Sedentario': 'n'})
)
conteo['count'] = round(conteo['count'] / conteo['count'].sum() * 100)
datos_dict = dict(zip(conteo['n'], conteo['count']))
fig = plt.figure(
    FigureClass=Waffle,
    rows=5,
    values=datos_dict,
    title={'label': 'Distribución de pacientes', 'loc': 'center'},
    colors=["#4CAF50", "#FFC107"],
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    icons='user', icon_size=20, icon_legend=True
)

plt.show()

### Gráfico treemap

In [ ]:
conteo = (
    datos['Sedentario']
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Sedentario', 'Sedentario': 'n'})
)

sizes = conteo['count']
labels = conteo['n'] + "\n" + conteo['count'].astype(str)
colors = plt.cm.Set2(range(len(sizes)))  # paleta de colores

plt.figure(figsize=(6, 4))
squarify.plot(
    sizes=sizes,
    label=labels,
    color=colors,
    text_kwargs={'fontsize':12, 'color':'white'},
    pad=True
)

## Gráficas para variables cuantitativas

### Gráfica de varas (barras)

In [ ]:
conteo = (
    datos.loc[datos['Edad'] > 60, 'Edad']
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={'index': 'Edad', 'Edad': 'n'}))

fig, ax = plt.subplots(figsize=(8, 4))

ax.bar(
    conteo['n'].astype(str),  # convertir a texto para eje x discreto
    conteo['count'],
    color='darkgreen',
    width=0.05)

for i, row in conteo.iterrows():
    ax.text(x=row['n'], y=row['count'] + 0.1, s=str(row['n']),
            ha='center', va='bottom', fontsize=9)

ax.set_ylim(0, 4.5)
ax.set_title("Distribución de las edades de los pacientes adultos mayores")
ax.set_xlabel("Valor")
ax.set_ylabel("Frecuencia")
plt.tight_layout()
plt.show()

### Histograma

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(datos['Edad'],
         bins=range(int(datos['Edad'].min()), int(datos['Edad'].max()) + 5, 5),
         color='mediumorchid', edgecolor='white')
plt.title("Distribución de la edad")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(data=datos, x='Edad',
             binwidth=5, color='mediumorchid', edgecolor='white')
plt.title("Distribución de la edad")
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

### Gráfico de densidad

In [ ]:
plt.figure(figsize=(8, 4))
sns.kdeplot(data=datos, x='Edad', fill=True, color='chocolate', alpha=0.4)
plt.title("Curva de densidad de la edad")
plt.xlabel("Edad")
plt.ylabel("Densidad")
plt.tight_layout()
plt.show()

### Boxplot

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='Edad', data=datos, color='lightblue', width=0.2)
plt.title("Distribución de la edad")
plt.ylabel("Edad")
plt.xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(4,6))
sns.boxplot(x='Sedentario', y='Edad', data=datos, color='lightblue')
plt.title("Distribución de la edad según actividad física")
plt.xlabel("Actividad física")
plt.ylabel("Edad")
plt.tight_layout()
plt.show()

### Gráfica de violín

In [ ]:
plt.figure(figsize=(4, 6))
sns.violinplot(y='Edad', data=datos, color='skyblue')
plt.title("Distribución de la edad")
plt.ylabel("Edad")
plt.xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
sns.violinplot(x = 'Sedentario', y='Edad', data=datos, color='skyblue')
plt.title("Distribución de la edad según actividad física")
plt.ylabel("Edad")
plt.xlabel("Actividad física")
plt.tight_layout()
plt.show()

# Resúmenes

In [ ]:
datos.describe(include='all').transpose()

In [ ]:
skim(datos)

In [ ]:
ProfileReport(datos, title="Resumen del DataFrame", explorative=True)